In [40]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [41]:
class MLP(nn.Module):
    def __init__(self, in_channels, hidden_layers, out_channels):
        super().__init__()
        layers = []
        prev_layer = in_channels

        for hidden_dim in hidden_layers:
            layers.append(nn.Linear(prev_layer, hidden_dim))
            layers.append(nn.ReLU())
            prev_layer = hidden_dim

        layers.append(nn.Linear(prev_layer, out_channels))
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        return self.mlp(x)

In [42]:
class GRUDecoder(nn.Module):
    def __init__(self, in_channels, out_channels, hidden_gru=64):
        super().__init__()
        hidden_layers = [128, 64, 32]

        self.gru = nn.GRU(in_channels, hidden_gru, batch_first=True)
        self.mlp = MLP(hidden_gru, hidden_layers, out_channels)

    def forward(self, z):
        gru_out, _ = self.gru(z)
        output = self.mlp(gru_out.squeeze())
        return output


In [43]:
class Model(torch.nn.Module):
    def __init__(self, input_size, hidden_channels, out_channels):
        super().__init__()
        self.encoder = nn.Linear(input_size, hidden_channels) #linear encoder
        self.decoder = GRUDecoder(hidden_channels, out_channels)

    def forward(self, x):
        z = self.encoder(x)
        z = z.unsqueeze(1)
        return self.decoder(z)

In [44]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [45]:
try:

    speed_data = pd.read_csv('dataset/PeMSD7_V_228.csv', header=None)

    adjacency_matrix = pd.read_csv('dataset/PeMSD7_W_228.csv', header=None)
except FileNotFoundError:
    import os
    import requests
    import zipfile


    url = "https://github.com/VeritasYin/STGCN_IJCAI-18/raw/master/dataset/PeMSD7_Full.zip"

    os.makedirs('dataset', exist_ok=True)
    response = requests.get(url)
    zip_path = 'dataset/PeMSD7_Full.zip'
    with open(zip_path, 'wb') as f:
        f.write(response.content)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('dataset')


    speed_data = pd.read_csv('dataset/PeMSD7_V_228.csv', header=None)


    adjacency_matrix = pd.read_csv('dataset/PeMSD7_W_228.csv', header=None)

In [46]:
scaler = StandardScaler()
speed_data = scaler.fit_transform(speed_data)

In [47]:
train_size = int(len(speed_data) * 0.8)
train_data = speed_data[:train_size]
test_data = speed_data[train_size:]

train_data = torch.tensor(train_data, dtype=torch.float32)
test_data = torch.tensor(test_data, dtype=torch.float32)

In [48]:
input_size = speed_data.shape[1]  # Number of sensors
hidden_channels = 64
out_channels = speed_data.shape[1]
num_epochs = 50
learning_rate = 0.001

In [49]:
model = Model(input_size, hidden_channels, out_channels)
optimizer = Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()


In [50]:
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Prepare input and target
    input_data = train_data[:-1]
    target_data = train_data[1:]

    # Forward pass
    output = model(input_data)
    loss = loss_fn(output, target_data)

    # Backward pass
    loss.backward()
    optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/50, Loss: 1.005492925643921
Epoch 2/50, Loss: 1.0036362409591675
Epoch 3/50, Loss: 1.002098560333252
Epoch 4/50, Loss: 1.000372290611267
Epoch 5/50, Loss: 0.9982897043228149
Epoch 6/50, Loss: 0.9958019256591797
Epoch 7/50, Loss: 0.9928260445594788
Epoch 8/50, Loss: 0.9893143773078918
Epoch 9/50, Loss: 0.9852292537689209
Epoch 10/50, Loss: 0.9804682731628418
Epoch 11/50, Loss: 0.9749297499656677
Epoch 12/50, Loss: 0.9685161113739014
Epoch 13/50, Loss: 0.9612706899642944
Epoch 14/50, Loss: 0.9532877802848816
Epoch 15/50, Loss: 0.9448174834251404
Epoch 16/50, Loss: 0.936309814453125
Epoch 17/50, Loss: 0.9281448721885681
Epoch 18/50, Loss: 0.9204778671264648
Epoch 19/50, Loss: 0.9130458235740662
Epoch 20/50, Loss: 0.9051369428634644
Epoch 21/50, Loss: 0.8960779309272766
Epoch 22/50, Loss: 0.8859115839004517
Epoch 23/50, Loss: 0.8749741315841675
Epoch 24/50, Loss: 0.8636137843132019
Epoch 25/50, Loss: 0.8520757555961609
Epoch 26/50, Loss: 0.8403916954994202
Epoch 27/50, Loss: 0.8284

In [51]:
model.eval()
with torch.no_grad():
    # Prepare input and target
    input_data = test_data[:-1]
    target_data = test_data[1:]

    # Forward pass
    output = model(input_data)

    # Calculate metrics
    mse = mean_squared_error(target_data.numpy(), output.numpy())
    mae = mean_absolute_error(target_data.numpy(), output.numpy())
    mape = mean_absolute_percentage_error(target_data.numpy(), output.numpy())
    rmse = np.sqrt(mse)

    print(f'MSE: {mse}')
    print(f'MAE: {mae}')
    print(f'MAPE: {mape}')
    print(f'RMSE: {rmse}')

MSE: 0.564700186252594
MAE: 0.4765739440917969
MAPE: 231.72047424316406
RMSE: 0.7514653593164451
